In [1]:
import _pickle as pickle
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import timeit

from tqdm.notebook import tqdm

In [2]:
stats_keys = [
    "accuracy",
    "p_iwl",
    "context contains query class",
    "ic_pred",
    "iw_pred",
    "num p_iwl >= 0.5",
    "p_iwl given context contains query class",
    "loss"
    # "similarity",
]

# Transformer
stats_keys = [
    "accuracy",
    "context contains query class",
    "loss"
]

In [3]:
# repo_path = "/Users/chanb/research/ualberta/icl/simple_icl"
# results_dir = "/Users/chanb/research/ualberta/icl/scratch/evaluation_results"
# results_dir = "/Users/chanb/research/ualberta/icl/cc_results/evaluation_results"
# results_dir = "/Users/chanb/research/ualberta/icl/cc_results/noisy_labels_0.01/evaluation_results"
# results_dir = "/Users/chanb/research/ualberta/icl/cc_results/paper_experiments/evaluation_results"
# results_dir = "/Users/chanb/research/ualberta/icl/simple_icl/experiments/test_convergence_eval"

# repo_path = "/home/chanb/src/simple_icl"
# results_dir = "/home/chanb/scratch/simple_icl/results"

repo_path = "/home/bryanpu1/projects/icl/simple_icl"
results_dir = "/home/bryanpu1/projects/icl/scratch/evaluation_results"

In [4]:
# variant_name = "synthetic-simple_icl"
# variant_name = "synthetic-transformer"
# variant_name = "synthetic-transformer-context_len"
# variant_name = "synthetic-transformer-large_num_low_freq"
# variant_name = "synthetic-transformer-large_num_low_freq-input_noise_0.2"
# variant_name = "synthetic-transformer-no_noise"
# variant_name = "synthetic-transformer-noisy_inputs_0.2"
# variant_name = "synthetic-transformer-noisy_labels_0.01"
# variant_name = "synthetic-transformer-noisy_labels_0.1"
# variant_name = "synthetic-transformer-num_relevant_contexts"

variant_name = "omniglot-input_noise"
# variant_name = "synthetic-transformer-noisy_inputs"
# variant_name = "synthetic-transformer-noisy_labels"
# variant_name = "synthetic-transformer-p_high"
# variant_name = "synthetic-transformer-p_relevant"
# variant_name = "synthetic-transformer-num_contexts"
# variant_name = "synthetic-transformer-num_low_freq"
# variant_name = "synthetic-transformer-num_relevant_contexts"
# variant_name = "synthetic-transformer-input_noise-longer_epoch"

os.makedirs(
    os.path.join(repo_path, "plot_utils/plots/agg_stats"),
    exist_ok=True
)

In [5]:
stats_file = os.path.join(repo_path, "plot_utils/plots/agg_stats", "{}.feather".format(variant_name))
if os.path.isfile(stats_file):
    stats = pd.read_feather(stats_file)
else:
    stats = []

    if variant_name == "synthetic-simple_icl":
        all_dirs = [
            "synthetic-alpha",
            "synthetic-ic_predictor",
            "synthetic-iw_predictor",
        ]
    else:
        all_dirs = [variant_name]

    for curr_dir in all_dirs:
        curr_results_dir = os.path.join(results_dir, curr_dir)
        if repo_path.startswith("/Users"):
            it = tqdm(os.listdir(curr_results_dir))
        else:
            it = os.listdir(curr_results_dir)

        for run_name in it:
            tic = timeit.default_timer()

            prefix = "-".join(run_name.split("-")[:-9])
            seed = run_name.split("-")[-9]

            data = pickle.load(open(os.path.join(curr_results_dir, run_name), "rb"))
            for eval_name in data["stats"]:
                for stats_key in stats_keys:
                    if not stats_key in data["stats"][eval_name]:
                        continue

                    factor = 1
                    if stats_key == "accuracy":
                        factor = 1 / 100

                    stats.append(
                        dict(
                            dirname=curr_dir,
                            variant=prefix,
                            seed=seed,
                            eval_name=eval_name,
                            stats_key=stats_key,
                            stats=[float(val) * factor for val in data["stats"][eval_name][stats_key]],
                            **{
                                "_".join(key_val.split("_")[:-1]): key_val.split("_")[-1] for key_val in prefix.split("-")
                            },
                        )
                    )

            toc = timeit.default_timer()
    stats = pd.DataFrame(stats)
    stats.to_feather(os.path.join(repo_path, "plot_utils/plots/agg_stats", "{}.feather".format(variant_name)))

In [6]:
len(stats)

1539

In [7]:
stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1539 entries, 0 to 1538
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   dirname             1539 non-null   object
 1   variant             1539 non-null   object
 2   seed                1539 non-null   object
 3   eval_name           1539 non-null   object
 4   stats_key           1539 non-null   object
 5   stats               1539 non-null   object
 6   dataset_size        1539 non-null   object
 7   p_relevant_context  1539 non-null   object
 8   input_noise_std     1539 non-null   object
dtypes: object(9)
memory usage: 108.3+ KB


In [8]:
stats

,dirname,variant,seed,eval_name,stats_key,stats,dataset_size,p_relevant_context,input_noise_std
0,omniglot-input_noise,dataset_size_10000-p_relevant_context_0.9-inpu...,seed_1,eval-default-none-flip_label,accuracy,"[0.0, 0.003, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",10000,0.9,0.1
1,omniglot-input_noise,dataset_size_10000-p_relevant_context_0.9-inpu...,seed_1,eval-default-none-flip_label,context contains query class,"[0.133, 0.133, 0.133, 0.133, 0.133, 0.133, 0.1...",10000,0.9,0.1
2,omniglot-input_noise,dataset_size_10000-p_relevant_context_0.9-inpu...,seed_1,eval-default-none-flip_label,loss,"[7.858643054962158, 7.570783615112305, 8.65740...",10000,0.9,0.1
3,omniglot-input_noise,dataset_size_10000-p_relevant_context_0.9-inpu...,seed_1,eval-default-high_prob-flip_label,accuracy,"[0.0, 0.009, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",10000,0.9,0.1
4,omniglot-input_noise,dataset_size_10000-p_relevant_context_0.9-inpu...,seed_1,eval-default-high_prob-flip_label,context contains query class,"[0.165, 0.165, 0.165, 0.165, 0.165, 0.165, 0.1...",10000,0.9,0.1
...,...,...,...,...,...,...,...,...,...
1534,omniglot-input_noise,dataset_size_1000000-p_relevant_context_1.0-in...,seed_1,eval-irrelevant_context-low_prob,context contains query class,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1000000,1.0,0.2
1535,omniglot-input_noise,dataset_size_1000000-p_relevant_context_1.0-in...,seed_1,eval-irrelevant_context-low_prob,loss,"[7.904811382293701, 8.27088737487793, 9.135965...",1000000,1.0,0.2
1536,omniglot-input_noise,dataset_size_1000000-p_relevant_context_1.0-in...,seed_1,pretraining,accuracy,"[0.0, 0.9298, 0.9794, 0.9997, 0.9999, 0.9706, ...",1000000,1.0,0.2
1537,omniglot-input_noise,dataset_size_1000000-p_relevant_context_1.0-in...,seed_1,pretraining,context contains query class,"[0.1606, 0.1606, 0.1606, 0.1606, 0.1606, 0.160...",1000000,1.0,0.2
